In [ ]:
from pathlib import Path

from document_ai.application.core.utils import pdf2image

REPO_DIR = Path().absolute().parent

filepath = REPO_DIR / "data/documents/deepseek-ocr-2.pdf"
output_dir = REPO_DIR / "data/output"

In [ ]:
images = pdf2image.convert_pdf_to_images(filepath)
image_path = REPO_DIR / "data/images/page.png"
image = images[0]
image.save(image_path)
image

In [ ]:
from transformers import AutoModel, AutoTokenizer
import torch
import os

os.environ["CUDA_VISIBLE_DEVICES"] = "0"
model_name = "deepseek-ai/DeepSeek-OCR-2"

tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
model = AutoModel.from_pretrained(
    model_name, trust_remote_code=True, use_safetensors=True
)
model = model.eval().cuda().to(torch.bfloat16)

In [ ]:
# prompt = "<image>\nFree OCR. "
prompt = "<image>\n<|grounding|>Convert the document to markdown. "
image_file = str(image_path)

res = model.infer(
    tokenizer,
    prompt=prompt,
    image_file=image_file,
    output_path=output_dir,
    base_size=1024,
    image_size=768,
    crop_mode=True,
    save_results=True,
)